<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/HMM2Vec/HMM2Vec_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hmmlearn

In [22]:
from hmmlearn import hmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# from google.colab import drive
# drive.mount('/content/drive')

In [23]:
df = pd.read_pickle('UnigramFilteredOpcodes.pkl')
opcodes_df = pd.read_csv('MostCommonOpcodes.csv')
#split dataset in half
df = df.sample(frac=0.5)

In [24]:
df.head()

Opcodes  MalwareType
2327  (lret, sub, lret, push, jne, jne, mov, sub, in...            2
1187  (mov, inc, and, and, add, in, sub, in, push, p...            1
540   (lea, mov, mov, inc, dec, dec, add, inc, add, ...            0
3112  (push, mov, mov, mov, mov, mov, pop, ret, push...            3
2805  (dec, dec, lret, in, inc, push, cmp, push, mov...            2

In [25]:
#creating a list of number representation for each opcode that is in the dataset
opcode_to_number = {}
count = 0
for opcode in opcodes_df['Opcodes']:
    opcode_to_number[opcode] = count
    count += 1

print(opcode_to_number)

{'add': 0, 'mov': 1, 'push': 2, 'pop': 3, 'inc': 4, 'xchg': 5, 'call': 6, 'or': 7, 'dec': 8, 'cmp': 9, 'xor': 10, 'sub': 11, 'and': 12, 'adc': 13, 'sbb': 14, 'lea': 15, 'test': 16, 'out': 17, 'in': 18, 'jmp': 19, 'movl': 20, 'int3': 21, 'ret': 22, 'imul': 23, 'je': 24, 'nop': 25, 'lods': 26, 'stos': 27, 'scas': 28, 'lret': 29, 'jne': 30}


In [26]:
def opcodes_to_numbers(columns):
  for sample in columns:
    # length = len(sample)
    # sequence_lengths.append(length)
    temp = []
    for opcode in sample:
      temp.append(opcode_to_number[opcode])
    opcode_sequences.append(temp)

In [27]:
#number of malware families
n_states = 4
#number of unique opcodes
n_obs = 31

In [28]:
opcode_sequences = []
opcodes_to_numbers(df['Opcodes'])

In [ ]:
#hmm for each sample
hmm_models = []
for opcode_seq in opcode_sequences:
  model = hmm.CategoricalHMM(n_components=n_states, n_iter=100)
  opcode_seq = np.array(opcode_seq)
  model.fit(opcode_seq.reshape(-1, 1))
  hmm_models.append(model)

In [ ]:
# Extract emission probabilities (B matrices)
B_matrices = [model.emissionprob_ for model in hmm_models]

# Vectorize B matrices
feature_vectors = []
for b_matrix in B_matrices:
  b_matrix = b_matrix.flatten()
  feature_vectors.append(b_matrix)

In [ ]:
print(B_matrices)

In [ ]:
#save results
np.save('half_HMM2Vec_embeddings.npy', feature_vectors)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
#split data
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, df['MalwareType'], test_size=0.2, random_state=42)

#train SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

#predict
y_pred = svm.predict(X_test)

#evaluate
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
